In [97]:
import pandas as pd #nul
import numpy as np
import missingno as msno
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import json
import branca.colormap as cm
import geopandas as gpd
import os
import re

import altair as alt
from altair import expr, datum
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import studentized_range
import scipy

In [ ]:
niveau_center = pd.read_excel(niveau_centre_path, header=4)

In [1]:
def mapped_mutations(df):

    loc_path = '../Data non localisée/'
    json_path = '../parcelles/'
    mutations = pd.DataFrame()


    for name_file in os.listdir(loc_path):
        print('df')
        df = pd.read_csv(loc_path+name_file)
        dept = name_file[11:13]
        print(dept)
        df['department_code'] = dept
        df['first_idpar'] = df.l_idpar.apply(lambda x: eval(x)[0])
        print('lecture')
        #json lecture
        with open(json_path + f'cadastre-{dept}-parcelles.json', 'r') as jsonFile:
            json_data = json.load(jsonFile)
        print('gdf')
        gdf = gpd.GeoDataFrame.from_features(json_data["features"])
        gdf = gdf[['geometry', 'id']].copy()
        print('merge')
        dfd = pd.merge(df, gdf, how='left', left_on='first_idpar', right_on='id')
        
        mutations = pd.concat([mutations, dfd])

In [100]:
mutations = pd.read_pickle('../data/geometric_mutations.pkl')

In [22]:
mutations = mutations[['idmutation', 'geometry', 'idmutinvar', 
       'idnatmut',  'datemut', 'valeurfonc',
       'l_codinsee','first_idpar', 'department_code']].copy()
# mutations = mutations[['idmutation', 'geometry', 'idmutinvar', 
#        'idnatmut',  'datemut', 'anneemut', 'moismut',
#     'libnatmut', 'vefa', 'valeurfonc', 'nblot',
#        'l_codinsee','first_idpar',
#        'nbparmut', 'sterr', 'nbvolmut', 'nblocmut',
#        'l_idlocmut', 'nblocmai', 'nblocapt', 'nblocdep', 'nblocact',
#        'nbapt1pp', 'nbapt2pp', 'nbapt3pp', 'nbapt4pp', 'nbapt5pp', 'nbmai1pp',
#        'nbmai2pp', 'nbmai3pp', 'nbmai4pp', 'nbmai5pp', 'sbati', 'sbatmai',
#        'sbatapt', 'sbatact', 'sapt1pp', 'sapt2pp', 'sapt3pp', 'sapt4pp',
#        'sapt5pp', 'smai1pp', 'smai2pp', 'smai3pp', 'smai4pp', 'smai5pp',
#        'codtypbien', 'libtypbien', 'department_code']].copy()

In [98]:
#### Geo dataframe
mutations = gpd.GeoDataFrame(mutations, geometry=mutations.geometry)
#### centroids
mutations['centroid'] = mutations.geometry.centroid
#### postcode 
mutations['l_codinsee'] = mutations.first_idpar.str[:5]
#### date data
mutations.datemut = pd.to_datetime(mutations.datemut)
mutations['month'] = mutations.datemut.dt.month
mutations['year'] = mutations.datemut.dt.year
mutations['day'] = mutations.datemut.dt.day
#### niveau center data
niveau_center = pd.read_excel('../data/niveau_centre.xlsx', header=4)
mutations = pd.merge(mutations, niveau_center[['codgeo', 'nivcentr']], how='left', left_on='l_codinsee', right_on='codgeo', right_index=False)
del mutations['codgeo']
#we consider that if no center level then 0
mutations.nivcentr = mutations.nivcentr.fillna(0)


In [99]:
mutations.

,idmutation,geometry,idmutinvar,idnatmut,datemut,valeurfonc,l_codinsee,first_idpar,department_code,centroid,month,year,day,nivcentr
0,6480289.0,"POLYGON ((1.92996 48.76034, 1.93036 48.76097, ...",cece6b3ada69e0498ac8b03997de97f3,2.0,2019-05-02,297300.0,78383,783830000W0078,78,POINT (1.93008 48.76068),5,2019,2,3.0
1,6479216.0,"POLYGON ((1.97224 48.87232, 1.97221 48.87233, ...",6eeb8a39a874ab39e090cbfac456dbb9,1.0,2019-10-22,100.0,78233,78233000AD0209,78,POINT (1.97222 48.87232),10,2019,22,1.0
2,6626911.0,"POLYGON ((2.13297 48.80721, 2.13287 48.80705, ...",ff7ff5f95678948b804cce5c33ced73c,1.0,2018-06-05,368000.0,78646,78646000AI0206,78,POINT (2.13300 48.80711),6,2018,5,4.0
3,6756560.0,"POLYGON ((2.02520 48.93726, 2.02559 48.93746, ...",ab4e4104a5fd3d9f44fe91d474e5f094,2.0,2017-06-16,170000.0,78123,78123000AM0709,78,POINT (2.02487 48.93782),6,2017,16,2.0
4,6627239.0,"POLYGON ((1.94849 48.82285, 1.94849 48.82285, ...",b6fb2b903e8ef3c816023f2289858072,1.0,2018-04-26,171000.0,78490,78490000AW0044,78,POINT (1.94882 48.82296),4,2018,26,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1271563,8369085.0,"POLYGON ((2.56393 48.86671, 2.56399 48.86690, ...",6230fc60f81376590d5225d2b05c5ff7,1.0,2016-09-15,183000.0,93032,93032000BR0322,93,POINT (2.56390 48.86687),9,2016,15,3.0
1271564,8279483.0,"POLYGON ((2.35700 48.93296, 2.35701 48.93294, ...",0f0d4aad8496ef69531d42c86db323fc,1.0,2017-03-30,200000.0,93066,93066000BI0070,93,POINT (2.35715 48.93290),3,2017,30,4.0
1271565,7998804.0,"POLYGON ((2.57483 48.94101, 2.57479 48.94107, ...",21954aa40d7d448f132b5f9e259532c9,2.0,2020-10-05,215000.0,93073,93073000AR0524,93,POINT (2.57453 48.94095),10,2020,5,3.0
1271566,8092334.0,"POLYGON ((2.55670 48.87797, 2.55671 48.87798, ...",e6567ba4d60c27f14c5fa67bcef55289,1.0,2019-06-28,135000.0,93032,93032000CE0188,93,POINT (2.55650 48.87801),6,2019,28,3.0
